In [2]:
from pymongo import MongoClient
import pprint

In [3]:

client = MongoClient('localhost', 27017)
db=client['reviews']
coll=db['user_reviews']

In [16]:
db.list_collection_names()



['user_reviews']

In [19]:
coll[3].user_reviews

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'reviews'), 'user_reviews.3.user_reviews')

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'reviews'), 'user_reviews.reviews')